In [1]:
from importlib import reload
import grammar
import io_tools

grammar = reload(grammar)
io_tools = reload(io_tools)

In [2]:
mode = "django"

In [3]:
indexer = grammar.Indexer(mode)

In [4]:
target_train_dataset = io_tools.json_to_tree_dataset("../../data/django_dataset/django.train.json",indexer)

In [5]:
type(target_train_dataset[0])

tree.OracleTree

In [6]:
import dynet as dy
import time
import random
import math
import sys
from argparse import ArgumentParser
from collections import Counter, defaultdict


In [7]:
import source_dataset
source_dataset = reload(source_dataset)
sourceDataset = source_dataset.SourceDataset(mode)

Using Django dataset to generate the indexer...
Source vocabulary size: 7926


In [8]:
vocab_length_source = sourceDataset.vocab_length
vocab_length_target = indexer.vocab_length
vocab_length_nodes = indexer.node_length
vocab_length_rules = indexer.rule_length

In [18]:
import tree
tree = reload(tree)

In [19]:
import model1
target_train_dataset = io_tools.json_to_tree_dataset("../../data/django_dataset/django.train.json",indexer)
args = model1.args
model1 = reload(model1)
net = model1.ASTNet(args=args, vocabLengthSource=vocab_length_source,
                       vocabLengthActionRule=vocab_length_rules, vocabLengthNodes=vocab_length_nodes, 
                       vocabLengthTarget=vocab_length_target)
trainer = dy.AdamTrainer(net.model)
net.model.populate("tmp.model")

In [20]:
def train():
    for i in range(0, len(target_train_dataset)):
        input_s, output_s = sourceDataset.train_index[i], target_train_dataset[i]
        output_s.set_to_root()
        output_s.set_query(input_s)
        # print(input_s,output_s.current_node)
        # print(type(output_s))
        all_losses = net.forward_prop(input_s, output_s, mode = "train")
        loss = dy.sum_batches(dy.esum(all_losses))
        loss.backward()
        trainer.update()

        if (i+1)%100==0:
            print(i+1,":",loss.value())
    net.model.save("tmp.model")
train()

100 : 23.412464141845703
200 : 18.471939086914062
300 : 39.91124725341797
400 : 0.0060596466064453125
500 : 23.73493766784668
600 : 11.579529762268066
700 : 49.8653564453125
800 : 23.06100082397461
900 : 29.828861236572266
1000 : 22.65554428100586
1100 : 8.984535217285156
1200 : 41.802528381347656
1300 : 26.173505783081055
1400 : 27.985639572143555
1500 : 38.709171295166016
1600 : 22.292112350463867
1700 : 0.007303714752197266
1800 : 16.796676635742188
1900 : 8.287311553955078
2000 : 7.871286869049072
2100 : 29.087202072143555
2200 : 17.089454650878906
2300 : 32.592750549316406
2400 : 9.142014503479004
2500 : 13.20496940612793
2600 : 0.009135007858276367
2700 : 202.919677734375
2800 : 44.103206634521484
2900 : 80.05491638183594
3000 : 10.027997016906738
3100 : 153.58480834960938
3200 : 165.8301544189453
3300 : 35.62496566772461
3400 : 81.61805725097656
3500 : 14.399227142333984
3600 : 26.941598892211914
3700 : 92.30233764648438
3800 : 41.28119659423828
3900 : 81.2020263671875
4000 : 17

In [ ]:
net.model.populate("tmp.model")
input_s = sourceDataset.train_index[0]
print(input_s)
output_t = tree.BuildingTree(indexer, input_s)
output_t = net.forward_prop(input_s, output_t, mode = "predict")

[2, 3, 4, 5, 6, 7, 8, 9, 10, 1]


In [93]:
print(output_t)

None
